In [5]:
import numpy as np
import pyvinecopulib as pv

def get_ced_cing(T, cop=None):
    d = T.shape[1]
    cing = []
    ced = []
    param = []
    for j in range(d):
        for i1 in range(d-j-1):
            ced.append(sorted((T[i1,j], T[d-j-1, j])))
            tmp = []
            for i2 in range(i1):
                tmp.append(T[i2,j])
            cing.append(sorted(tmp))
            if cop is not None:
                param.append(cop.get_parameters(i1, j)[0][0])
    return ced, cing, param

def find(a, cing_len, ced, cing):
    out = [i for i in range(len(ced)) if a in ced[i]]
    matched = False
    for i in out:
        if len(cing[i]) == cing_len:
            matched = True
            break
    assert matched, f'bad argument, a={a}, cing_len={cing_len}'
    return ced[i][1] if ced[i][0] == a else ced[i][0]

def diagonalize(T1, a):
    d = T1.shape[1]
    if a == T1[d-1, 0]:
        return T1
    assert a == T1[d-2, 0], f'cannot be diagonalized with {a}'

    T2 = np.zeros(shape=T1.shape, dtype=np.uint64)
    T2[d-1, 0] = a
    order = [a]

    ced, cing, _ = get_ced_cing(T1)
    for j in range(d-1):
        for i in range(d-j-1):
            T2[i,j] = find(T2[d-j-1, j], i, ced, cing)

        remove_idx = [i for i, c in enumerate(ced) for k in order if k in c]
        keep_idx = set(range(len(ced))).difference(set(remove_idx))

        ced = [ced[i] for i in keep_idx]
        cing = [cing[i] for i in keep_idx]

        T2[d-j-2, j+1] = T2[d-j-2, j]
        order.append(T2[d-j-2, j+1])
    return T2

In [32]:
T = np.array([[2, 2, 1, 1, 1],
[1, 1, 2, 2, 0],
[3, 3, 3, 0, 0], 
[4, 4, 0, 0, 0],
[5, 0, 0, 0, 0]])
struct = pv.RVineStructure(T)

In [33]:
T

array([[2, 2, 1, 1, 1],
       [1, 1, 2, 2, 0],
       [3, 3, 3, 0, 0],
       [4, 4, 0, 0, 0],
       [5, 0, 0, 0, 0]])

In [34]:
k = 1
d = T.shape[1]
for i in range(d):
    for j in range(d):
        if (i <= j) and (T[i,j] != 0):
            print(f'({i},{j})', T[i][j])
            k += 1

(0,0) 2
(0,1) 2
(0,2) 1
(0,3) 1
(0,4) 1
(1,1) 1
(1,2) 2
(1,3) 2
(2,2) 3


In [25]:
k

10

In [28]:
T

array([[2, 2, 1, 1, 1],
       [1, 1, 2, 2, 0],
       [3, 3, 3, 0, 0],
       [4, 4, 0, 0, 0],
       [5, 0, 0, 0, 0]])

In [27]:
np.count_nonzero(T!=0)

15

In [18]:
diagonalize(T, 4)

array([[2, 2, 1, 1, 1],
       [1, 1, 2, 2, 0],
       [3, 3, 3, 0, 0],
       [5, 5, 0, 0, 0],
       [4, 0, 0, 0, 0]], dtype=uint64)